In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam


model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Conv1D(filters=128, kernel_size=2, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])


Unknown OS detected


SystemExit: 1

/Users/asger/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import os

# Ensure the directory exists
if not os.path.exists('stock_dfs'):
    os.makedirs('stock_dfs')

# Now you can save your file to 'stock_dfs'

In [3]:
# Downloading all stocks in S&P 500 from yahoo finance 

import yfinance as yf
import pandas as pd
import datetime

# S&P 500 stock tickers
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500.to_csv('sp500.csv')
sp500 = pd.read_csv('sp500.csv', header=0)
sp500 = sp500['Symbol'].values.tolist()

# Downloading stock data from the last 10 years untill today 13/03-2024
start = datetime.datetime(2014, 3, 13)
end = datetime.datetime(2024, 3, 13)

# Ensure the directory exists
if not os.path.exists('stock_dfs'):
    os.makedirs('stock_dfs')

for ticker in sp500:
    if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
        df = yf.download(ticker, start=start, end=end)
        df.to_csv('stock_dfs/{}.csv'.format(ticker))
    else:
        print('Already have {}'.format(ticker))

# Merging all stock data into one dataframe
main_df = pd.DataFrame()

for count, ticker in enumerate(sp500):
    df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
    df.set_index('Date', inplace=True)

    df.rename(columns={'Adj Close': ticker}, inplace=True)
    df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace=True)

    if main_df.empty:
        main_df = df
    else:
        main_df = main_df.join(df, how='outer')

main_df.to_csv('sp500_joined_closes.csv')

# Preprocessing the data
import numpy as np

# Load the data
data = pd.read_csv('sp500_joined_closes.csv')
data = data.set_index('Date')
data = data.dropna(axis=0)

data.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

KeyboardInterrupt: 